In [ ]:
!pip install langchain

In [ ]:
!pip install langchain-huggingface

In [ ]:
!pip install pypdf

In [ ]:
!pip install faiss-gpu

In [ ]:
!pip install ctransformers

In [ ]:
!pip install langchain_community

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install python-box

In [9]:
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
import box
from langchain import PromptTemplate
from langchain.chains import RetrievalQA


In [10]:
model_file="/content/drive/MyDrive/llama-2-7/llama-2-7b-chat.ggmlv3.q8_0.bin"

In [11]:
dir=DirectoryLoader("/content/",glob="**/*.pdf",loader_cls=PyPDFLoader)

In [12]:
dir_load=dir.load()

In [ ]:
dir_load

In [54]:
splitter_text=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=50)

In [55]:
docs= splitter_text.split_documents(dir_load)

In [56]:
docs

[Document(metadata={'source': '/content/drive/MyDrive/Bank_Questions_and_Answers_Corrected.pdf', 'page': 0}, page_content="Bank-Related Questions and Answers\nQ: What is a credit card loan?\nA: A credit card loan is a form of borrowing where you use your credit card to access funds, either\nthrough a cash advance or by purchasing items. The loan must be repaid with interest, and it\ntypically has higher interest rates compared to other types of loans.\nQ: What is a loan agreement?\nA: A loan agreement is a legal contract between a borrower and a lender that outlines the terms and\nconditions of the loan, including the repayment schedule, interest rate, and any penalties for late\npayment.\nQ: What documents are required for getting a loan?\nA: The documents required for a loan vary by lender, but commonly include:\n1. Proof of identity (e.g., passport, driver's license).\n2. Proof of income (e.g., pay stubs, tax returns).\n3. Bank statements.\n4. Proof of address (e.g., utility bills).

In [57]:
#EMBEDDING

embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device":"cpu"})

In [58]:
#embed the document and store in vector db

vector=FAISS.from_documents(docs,embedding)

In [59]:
vector.save_local("db/faiss")

In [60]:
#setting up model
llm=CTransformers(
    model=model_file,
    model_type="llama",
    config={"max_new_tokens":1000,"temperature":0.01}
)

In [61]:
#making prompt template

prompt="""
You are a bank chatbox, provide information only based on the context

Answer only for bank related queries.Suppose the question is out of context
respond out of context.

Context:{context}
Question:{question}
"""

In [62]:
def sq_prompt():
  prompt_template=PromptTemplate(template=prompt,input_variables=["context","question"])
  return prompt_template

In [63]:
def answer(llm_mode_type,prompt_type,v_db):
  ques_ans=RetrievalQA.from_chain_type(
      llm=llm_mode_type,
      chain_type="stuff",
      chain_type_kwargs={"prompt":prompt_type},
      retriever=v_db.as_retriever(search_kwargs={"k":2})

  )
  return ques_ans

In [64]:
def setup_model():
  embbed=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device":"cpu"})
  vdb=FAISS.load_local("db/faiss",embbed,allow_dangerous_deserialization=True)
  pmt=sq_prompt()
  get_ans=answer(llm,pmt,vdb)
  return get_ans


In [65]:
#final load run
llm_qa_model=setup_model()

In [66]:
llm_qa_model

RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\nYou are a bank chatbox, provide information only based on the context\n\nAnswer only for bank related queries.Suppose the question is out of context\nrespond out of context.\n\nContext:{context}\nQuestion:{question}\n'), llm=CTransformers(client=<ctransformers.llm.LLM object at 0x791084be7790>, model='/content/drive/MyDrive/llama-2-7/llama-2-7b-chat.ggmlv3.q8_0.bin', model_type='llama', config={'max_new_tokens': 1000, 'temperature': 0.01}), output_parser=StrOutputParser(), llm_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_variable_name='context'), retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.ve

In [52]:
asking_questions="What is a personal loan?"
llm_qa_model({"query":asking_questions})

{'query': 'What is a personal loan?',
 'result': 'Answer: A personal loan is an unsecured loan that can be used for various purposes, such as medical expenses, travel, or home renovation. It does not require collateral and is typically repaid in fixed monthly installments over a set period.'}

In [67]:
asking_questions=["What is the difference between secured and unsecured loans?",
"What is EMI?",
"What is the difference between fixed and floating interest rates?",
"What is a loan-to-value (LTV) ratio?"]

for question in asking_questions:
  response=llm_qa_model({"query":question})
   #Extract the answer (assuming `result` contains the actual answer)
  answer = response.get("result", "No answer provided")


    # Clean the answer by removing unwanted 'A:' prefix if it exists
  if answer.startswith("A:"):
    answer = answer[2:].strip()  # Remove the "A:" part and any extra spaces

  print(f"Question:{question}")
  print(f"Answer:{answer}")
  print("*********************\n")


KeyboardInterrupt: 